In [1]:
#exp8 text classification on reviews dataset
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
textdata = pd.read_csv('reviews.csv')

In [3]:
textdata.shape

(386, 2)

In [4]:
textdata=textdata.drop_duplicates()

In [5]:
textdata.shape

(131, 2)

In [6]:
textdata.head()

Review Sentiment
0  This product exceeded my expectations! It's hi...  Positive
1  The product was decent. It worked fine, but it...   Neutral
2  I had a terrible experience with this company....  Negative
3  It's an okay product. Nothing to write home ab...   Neutral
4  Disappointed with the product. It didn't meet ...  Negative

In [7]:
data = textdata.iloc[:, 0]
target = textdata.iloc[:,1]

In [8]:
target.value_counts()

Sentiment
Positive    45
Negative    44
Neutral     42
Name: count, dtype: int64

In [9]:
X_train_text, X_test_text, y_train, y_test = train_test_split(data, target,
                                                              random_state = 42)

In [10]:
vect = CountVectorizer(min_df = 5, ngram_range=(2,3))
vect.fit(X_train_text)
X_train = vect.transform(X_train_text)
X_test = vect.transform(X_test_text)

In [11]:
repr(X_train)

"<Compressed Sparse Row sparse matrix of dtype 'int64'\n\twith 180 stored elements and shape (98, 25)>"

In [12]:
feature_names = vect.get_feature_names_out()

In [13]:
feature_names[::10]

array(['and the', 'nor bad', 'this company'], dtype=object)

In [14]:
scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)
print(scores)

[0.7        0.6        0.55       0.73684211 0.52631579]


In [15]:
np.mean(scores)

0.6226315789473683

In [16]:
param_grid = { 'C':[0.0001, 0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print('best cross validation score : ', grid.best_score_)
print('best parameters : ', grid.best_params_)

best cross validation score :  0.6226315789473684
best parameters :  {'C': 0.1}


In [17]:
lr = LogisticRegression(C=1)
lr.fit(X_train, y_train)
print('training score : ', lr.score(X_train, y_train))
print('testing score : ', lr.score(X_test, y_test))

training score :  0.7142857142857143
testing score :  0.6363636363636364


In [18]:
def try_review(review: str):
    review_text = pd.Series(review)
    review = vect.transform(review_text)
    result = lr.predict(review)
    print(*result)

In [19]:
try_review('very good product')

Positive


In [20]:
try_review('The service was satisfactory, nothing good')

Neutral


In [21]:
mnc = MultinomialNB()
mnc.fit(X_train, y_train)
print(mnc.score(X_train, y_train))
print(mnc.score(X_test, y_test))

0.6938775510204082
0.6363636363636364


In [22]:
def try_mnc(review: str):
    review_text = pd.Series(review)
    review = vect.transform(review_text)
    result = mnc.predict(review)
    print(*result)

In [23]:
try_mnc('The product is outstanding')

Positive
